# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming


## Install required packages

In [ ]:
!pip install tensorflow pandas nltk numpy matplotlib scikit-learn sentencepiece tokenizers
!python -m spacy download en_core_web_sm


In [1]:
import pandas as pd
import regex as re
import numpy as np
import nltk
import os
import tensorflow as tf
import spacy

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from tokenizers import ByteLevelBPETokenizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
print("GPUs:", tf.config.list_physical_devices('GPU'))

nlp = spacy.load("en_core_web_sm")

GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
from google.colab import drive
drive.mount('/content/drive')
if not os.path.exists('glove_embeddings'):
  !mkdir glove_embeddings
  !cp /content/drive/MyDrive/glove.6B.200d.txt glove_embeddings/


Mounted at /content/drive


In [ ]:
if not os.path.exists('glove_embeddings'):
  !wget http://nlp.stanford.edu/data/glove.6B.zip -O glove.zip
  !unzip -j glove.zip "glove.6B.200d.txt" -d glove_embeddings

--2025-03-25 22:33:25--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-03-25 22:33:26--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-25 22:33:26--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.zip’

glove.z

## Load dataset

In [5]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [6]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1.0
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1.0
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1.0
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1.0
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1.0


In [7]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [8]:
dev_set['premise_tokens'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_tokens'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_tokens'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_tokens'] = train_set['hypothesis'].apply(clean_text)

In [9]:
dev_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]"
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]"
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei..."
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l..."
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]"


In [10]:
train_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1.0,"[yeah, know, cut, california, half, something]","[yeah, sure, make, fit, maybe, could, cut, cal..."
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1.0,"[actual, name, used]","[sake, privacy, actual, name, used]"
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1.0,"[film, directed, randall, wallace]","[film, directed, randall, wallace, star, mel, ..."
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1.0,"[know, sign, anse, studied, unkempt, clean, re...","[anse, looked, cracked, mirror]"
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1.0,"[light, candle, cheek, looked, even, hollow, t...","[drew, regarded, best, friend, noted, light, l..."


Dataset analysis

# NER (might use)


In [11]:

def ner_features(text1, text2):
    doc1, doc2 = nlp(text1), nlp(text2)
    ents1 = set(ent.text.lower() for ent in doc1.ents)
    ents2 = set(ent.text.lower() for ent in doc2.ents)
    overlap = len(ents1 & ents2) / (len(ents1 | ents2) + 1e-5)
    return np.array([overlap])


# TF-IDF word embeddings

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert tokenized text back to strings for TF-IDF processing
train_premise_texts = train_set['premise_tokens'].apply(lambda x: ' '.join(x))
train_hypothesis_texts = train_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

dev_premise_texts = dev_set['premise_tokens'].apply(lambda x: ' '.join(x))
dev_hypothesis_texts = dev_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

# Combine premises & hypotheses for TF-IDF training
all_texts = pd.concat([train_premise_texts, train_hypothesis_texts])

# Initialize and fit TF-IDF on training data
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf.fit(all_texts)

# Transform training and dev sets
train_premise_tfidf = tfidf.transform(train_premise_texts).toarray()
train_hypothesis_tfidf = tfidf.transform(train_hypothesis_texts).toarray()

dev_premise_tfidf = tfidf.transform(dev_premise_texts).toarray()
dev_hypothesis_tfidf = tfidf.transform(dev_hypothesis_texts).toarray()



# Glove embeddings

In [13]:
glove = "./glove_embeddings/glove.6B.200d.txt"
def load_glove(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)  # <-- Convert to float32
            embeddings_dict[word] = vector
    return embeddings_dict

embedding_dim = 200
loaded_glove = load_glove(glove)



In [14]:
from sklearn.metrics.pairwise import cosine_similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-5)


In [15]:
#Traidntiona emmedings
"""
def sentence_embedding(tokens, embeddings_dict, embedding_dim):
    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]
    if not valid_embeddings:
        return np.zeros(embedding_dim)
    return np.mean(valid_embeddings, axis=0)

def pairwise_embedding(premise_tokens, hypothesis_tokens,  embeddings_dict,embedding_dim):
    prem_emb = sentence_embedding(premise_tokens, embeddings_dict, embedding_dim)
    hyp_emb = sentence_embedding(hypothesis_tokens, embeddings_dict, embedding_dim)
    ner_feat = ner_features(train_set['premise'], train_set['hypothesis'])
    cos_sim = np.array([cosine_similarity(prem_emb, hyp_emb)])

    combined = np.concatenate([
        prem_emb,
        hyp_emb,
        np.abs(prem_emb - hyp_emb),
        ner_feat,
        cos_sim
    ]).astype(np.float32)

    return combined

"""

"\ndef sentence_embedding(tokens, embeddings_dict, embedding_dim):\n    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]\n    if not valid_embeddings:\n        return np.zeros(embedding_dim)\n    return np.mean(valid_embeddings, axis=0)\n\ndef pairwise_embedding(premise_tokens, hypothesis_tokens,  embeddings_dict,embedding_dim):\n    prem_emb = sentence_embedding(premise_tokens, embeddings_dict, embedding_dim)\n    hyp_emb = sentence_embedding(hypothesis_tokens, embeddings_dict, embedding_dim)\n    ner_feat = ner_features(train_set['premise'], train_set['hypothesis'])\n    cos_sim = np.array([cosine_similarity(prem_emb, hyp_emb)])\n\n    combined = np.concatenate([\n        prem_emb,\n        hyp_emb,\n        np.abs(prem_emb - hyp_emb),\n        ner_feat,\n        cos_sim\n    ]).astype(np.float32)\n\n    return combined\n\n"

In [16]:
"""
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Convert list of numpy arrays into a single 2D numpy array
train_embeddings = np.vstack(train_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))
dev_embeddings = np.vstack(dev_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))
train_labels = train_set['label'].values
dev_labels = dev_set['label'].values


train_embeddings_tf = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)
dev_embeddings_tf = tf.convert_to_tensor(dev_embeddings, dtype=tf.float32)

train_labels_tf = tf.convert_to_tensor(train_labels, dtype=tf.int32)
dev_labels_tf = tf.convert_to_tensor(dev_labels, dtype=tf.int32)
"""

"\nimport numpy as np\nfrom tqdm import tqdm\ntqdm.pandas()\n\n# Convert list of numpy arrays into a single 2D numpy array\ntrain_embeddings = np.vstack(train_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))\ndev_embeddings = np.vstack(dev_set.apply(lambda row: create_embedding(row, loaded_glove, embedding_dim), axis=1))\ntrain_labels = train_set['label'].values\ndev_labels = dev_set['label'].values\n\n\ntrain_embeddings_tf = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)\ndev_embeddings_tf = tf.convert_to_tensor(dev_embeddings, dtype=tf.float32)\n\ntrain_labels_tf = tf.convert_to_tensor(train_labels, dtype=tf.int32)\ndev_labels_tf = tf.convert_to_tensor(dev_labels, dtype=tf.int32)\n"

# tokinizer LSTM tokinizer

In [30]:
def tokens_to_embedding_sequence(tokens, glove, dim)

    return [glove.get(tok, np.zeros(dim)) for tok in tokens]

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def create_sequence_embedding(row, glove, dim, max_len):
    prem_seq = tokens_to_embedding_sequence(row['premise_tokens'], glove, dim)
    hyp_seq = tokens_to_embedding_sequence(row['hypothesis_tokens'], glove, dim)
    combined_seq = prem_seq + hyp_seq
    if len(combined_seq) == 0:
        print(f"Empty sequence found for row {row.name}")
        return np.zeros((max_len, dim))  # Return an empty sequence (zeroes) if no tokens are found

    padded_seq = pad_sequences([combined_seq], maxlen=max_len, dtype='float32', padding='post', truncating='post')[0]

    return padded_seq



In [44]:
max_len = 60
print("train")
train_seq = np.stack(train_set.apply(lambda row: create_sequence_embedding(row, loaded_glove, embedding_dim, max_len), axis=1))
print("dev")
dev_seq = np.stack(dev_set.apply(lambda row: create_sequence_embedding(row, loaded_glove, embedding_dim, max_len), axis=1))


train
Empty sequence found for row 4101
Empty sequence found for row 6441
Empty sequence found for row 6728
Empty sequence found for row 12057
dev
Empty sequence found for row 2397
Empty sequence found for row 4515


In [37]:
train_seq.shape

(17990, 50, 200)

#LSTM RNN appaorch


In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Bidirectional(LSTM(128, return_sequences=False), input_shape=(max_len, embedding_dim)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 256)                 │         336,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 353,474 (1.35 MB)

 Trainable params: 353,474 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model.fit(train_seq, train_set['label'].values, validation_data=(dev_seq, dev_set['label'].values), epochs=10, batch_size=64)



Epoch 1/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.5148 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 2/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4754 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 3/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4764 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 4/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4806 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 5/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4835 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 6/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4812 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 7/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4794 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 8/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4842 - loss: nan - val_accuracy: 0.4837 - val_loss: nan
Epoch 9/10
282/2

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

preds = model.predict(dev_embeddings_tf)
pred_labels = np.argmax(preds, axis=1)

print(classification_report(dev_labels_tf, pred_labels, target_names=['entailment', 'contradiction']))


# Traditional Approach

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

"""
param_grid = {
    'C': [0.01, 0.1, 0.2],  # Regularization strength
    'solver': ['lbfgs','saga','newton-cg','sag' ],  # Different solvers for logistic regression
    'max_iter': [500, 1000]  # More iterations for convergence
}


scaler = StandardScaler()
train_emb_scaled = scaler.fit_transform(train_embeddings)
dev_emb_scaled = scaler.transform(dev_embeddings)

param_grid = {
    'C': [0.2,0.1],  # Regularization strength
    'solver': ['saga','newton-cg','sag' ],  # Different solvers for logistic regression
    'max_iter': [200, 300, ]  # More iterations for convergence
}

clf = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
#clf = (solver='lbfgs', 'C'= 0.01, max_iter=500, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
clf.fit(train_emb_scaled, train_set["label"].values)  # Train on enhanced embeddings

# Print Best Parameters
print("Best Parameters:", clf.best_params_)

# Evaluate on validation set
preds = clf.best_estimator_.predict(dev_emb_scaled)
print(classification_report(dev_set['label'].values, preds, target_names=['entailment', 'contradiction']))
"""
